# Представление данных в компьютере

Ядро науки о данных (*Data Science*) и машинного обучения - **данные**: мы заинтересованы в извлечении знаний из данных. Но как именно компьютеры представляют данные? Давайте точно выясним, чему располагает «искусственный интеллект» для изучения.

## Данные представленые в виде массивов

Давайте посмотрим на некоторые фрукты. Используя библиотеку `Images.jl`, мы можем загрузить несколько изображений:

In [ ]:
using Images

cd("D:/Science") # load не любит кириллицу в пути, 
# а так как мой профиль назван по-русски, то придётся перенести папку и поменять рабочую директорию
apple = load("data/10_100.jpg")

In [ ]:
banana = load("data/104_100.jpg")

Здесь у нас есть изображения яблок и бананов. В конечном итоге мы хотели бы создать программу, которая может автоматически различать их. Однако компьютер не «видит» яблоко или банан; вместо этого он просто видит цифры. 

Изображение кодируется во что-то, называемое **массивом**, который похож на контейнер, в котором есть поля или ячейки для отдельных фрагментов данных:

![attachment:array_cartoon.png](data/array_cartoon.png)

Массив - это набор чисел в связанных блоках; На рисунке выше показан одномерный массив. Наши изображения вместо этого представляют собой двумерные массивы или матрицы чисел, расположенные примерно так:

![attachment:array2d.png](data/array2d.png)

Например, `apple` - это изображение, состоящее из массива чисел 100x100:

In [ ]:
typeof(apple)

In [ ]:
a = [ 1 2 3;4 5 6]

In [ ]:
typeof(a)

In [ ]:
size(a)

In [ ]:
size(apple)

Мы можем получить данные, хранящиеся в блоке в строке `i` и столбце` j`, путем * индексации * с использованием квадратных скобок: `[i, j]`. Например, давайте возьмем пиксель (часть изображения) в блоке $ (40, 60) $, то есть в 40-й строке и 60-м столбце изображения:

In [ ]:
apple

In [ ]:
dump(typeof(apple[40, 60]))

In [ ]:
apple[18:20,29:31] # 9-ть пикселей от яблока

Мы видим, что Юля показывает цветную коробку! Юлия, благодаря пакету «Colors.jl», достаточно умна, чтобы отображать цвета так, чтобы это было понятно для нас, людей! 

Таким образом, на самом деле изображение - это двумерный массив, в котором каждый элемент массива представляет собой объект (набор чисел), описывающий цветной пиксель.

## Цвета как числа

Как же тогда эти цвета на самом деле хранятся? Компьютеры хранят цвета в формате RGB, то есть они хранят значения от 0 до 1 для каждого из трех «каналов»: красного, зеленого и синего. Здесь 0 означает ни один из этого цвета, а 1 означает самую яркую форму этого цвета. Общий цвет представляет собой комбинацию этих трех цветов. Например, мы можем извлечь значение `red`, используя функцию` red`, примененную к цвету. 

Поскольку внутреннее фактическое значение хранится в специальном формате, мы решили преобразовать его в стандартное число с плавающей запятой, используя функцию `Float64`:

In [ ]:
Float64(red(apple[40, 60]))

In [ ]:
using Plots, Statistics
plotly() # у меня gr() сломался

In [ ]:
[ mean(float.(c.(img))) for c = [red,green,blue], img = [apple,banana] ]

In [ ]:
histogram(float.(green.(apple[:])),color="red",label="apple", normalize=true, nbins=25)
histogram!(float.(green.(banana[:])),color="yellow",label="banana",normalize=true, nbins=25)

In [ ]:
apple

In [ ]:
float(red(banana[50,20]))

In [ ]:
banana[50,20]

In [ ]:
pixel = apple[40, 60]

red_value   = Float64( red(pixel) )
green_value = Float64( green(pixel) )
blue_value  = Float64( blue(pixel) )

print("The RGB values are ($red_value, $green_value, $blue_value)")

Поскольку значение красного цвета высокое, а остальные низкие, это означает, что в пикселе `(40, 60)` изображение яблока очень красное. Если мы сделаем то же самое в одном из углов изображения, мы получим следующее:

In [ ]:
pixel = apple[1, 1]

red_value   = Float64( red(pixel) )
green_value = Float64( green(pixel) )
blue_value  = Float64( blue(pixel) )

print("The RGB values are ($red_value, $green_value, $blue_value)")

In [ ]:
apple

Мы видим, что каждый цвет яркий, что соответствует белому.

## Работа над изображением в целом

В Julia, чтобы применить функцию ко всему массиву, мы помещаем `.` между именем функции и левой круглой скобкой "`(`", поэтому следующее дает нам значение 'red' каждого пикселя в изображении:

In [ ]:
redpartofapple = Float64.(red.(apple))
mean(redpartofapple)

In [ ]:
histogram(redpartofapple[:],color=:red,label="Распределение красноты")

Обратите внимание, что мы получаем на выходе 2D массив (матрицу).

[Математическая стандартная библиотека Джулии] (https://docs.julialang.org/en/stable/stdlib/math/Matmatics-1) имеет много встроенных математических функций. Одной из них является функция `mean` (пакет `Statistics`), которая вычисляет среднее значение. Если мы применим это к нашему яблоку:

In [ ]:
mean(Float64.(red.(apple)))

мы видим, что значение указывает на то, что среднее количество красного на изображении является значением между количеством красного в яблоке и количеством красного на белом фоне.

*Каким-то образом нам нужно научить компьютер использовать эту информацию о картине, чтобы распознать, что там есть яблоко!*

## Небольшая загадка

Из научного интереса, давайте проверим среднее значение красного цвета на изображении банана.

In [ ]:
mean(Float64.(red.(banana)))

Не уж-то этот банан краснее чем наше яблоко? Это не ошибка и на самом деле это правда! Прежде чем перейти к следующему упражнению, внимательно изучите изображения яблок и бананов и посмотрите, сможете ли вы объяснить, почему этот результат вполне ожидаем.

#### Упражнение 1

Найдите среднее уровня синего в банане?

(Чтобы открыть новую ячейку, используйте <ESC>+b (b от "below", как думаете, что он делает?))

#### Упражнение 2

В банане больше синего или зеленого?